In [1]:
import ast
import pandas as pd
import numpy as np
from collections import OrderedDict
from perceptronac.loading_and_saving import base_colors
import matplotlib.colors as mcolors
from perceptronac.loading_and_saving import plot_comparison
from perceptronac.loading_and_saving import linestyle_tuple

In [2]:
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "legend.fontsize": 14,
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
    "pgf.preamble": [
        r"\usepackage{siunitx}",
        r"\usepackage{inputenc}",
        r"\DeclareUnicodeCharacter{2212}{\ensuremath{-}}",
        r"\DeclareUnicodeCharacter{03BC}{\ensuremath{\mu}}",
#         r"\newcommand{muJ}{\ensuremath{\mu J}}"
        
    ]})

/usr/lib/python3.9/_collections_abc.py:941: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [3]:

def custom_df_to_dict(in_df):

    out_dict = dict()
    for hsh in set(str(in_df[c].values.tolist()) for c in in_df.columns):
        lbls = []
        for c in in_df.columns:
            if str(in_df[c].values.tolist()) == hsh:
                lbls.append(c)
        out_dict[",".join(lbls)] = ast.literal_eval(hsh.replace("nan","None"))
    
    return out_dict

In [4]:
def line_func(k):
    
    lines = list(OrderedDict({
        'densely dashed':linestyle_tuple['densely dashed'],
        'solid':linestyle_tuple['solid'],
        'long dash with offset': linestyle_tuple['long dash with offset'],
        'densely dashdotdotted': linestyle_tuple['densely dashdotdotted'], # never used
        'dotted':linestyle_tuple['dotted'],
        'dashdotdotted': linestyle_tuple['dashdotdotted'],
        'dashdot':linestyle_tuple['dashdot'],
        'dashed2':linestyle_tuple['dashed2']
        ,
    }).values())
    
    if k == "Sobol":
        lin3 = lines[5]
    elif k == "qNEHVI":
        lin3 = lines[6]
    elif k == "qNParEGO":
        lin3 = lines[7]
    elif "Alg.2" in k:
        lin3 = lines[0]
    elif "Alg.3" in k:
        lin3 = lines[1]
    elif "Alg.4" in k:
        lin3 = lines[2]
    elif "Alg.5" in k:
        lin3 = lines[3]
    elif "Alg.6" in k:
        lin3 = lines[4]
    elif "Max HV" in k:
        lin3 = lines[3]
#     print(k,"line",lin3)
    return lin3

def color_func(k):
    
#     colors1 = list(base_colors.keys())
#     colors2 = list(mcolors.TABLEAU_COLORS.keys())
    
#     colors = colors1[:7] + colors2[7:]
    
    colors = list(OrderedDict({
#         'b': base_colors['b'],
#         'g': base_colors['g'],
#         'r': base_colors['r'],
#         'c': base_colors['c'],
#         'm': base_colors['m'],
#         'y': base_colors['y'],
#         'k': base_colors['k'],
#         'w': base_colors['w'],
        'tab:red': mcolors.TABLEAU_COLORS['tab:red'], 
        'tab:green': mcolors.TABLEAU_COLORS['tab:green'],
        'tab:blue': mcolors.TABLEAU_COLORS['tab:blue'],
        'tab:purple': mcolors.TABLEAU_COLORS['tab:purple'], # never used
        'k': base_colors['k'],
        'tab:orange': mcolors.TABLEAU_COLORS['tab:orange'],
#         'tab:brown': mcolors.TABLEAU_COLORS['tab:brown'],
        'tab:pink': mcolors.TABLEAU_COLORS['tab:pink'],
        'tab:gray': mcolors.TABLEAU_COLORS['tab:gray'],
        'tab:olive': mcolors.TABLEAU_COLORS['tab:olive'],
        'tab:cyan': mcolors.TABLEAU_COLORS['tab:cyan']        
    }).values())
    

    if k == "Sobol":
        color = colors[5]
    elif k == "qNEHVI":
        color = colors[6]
    elif k == "qNParEGO":
        color = colors[7]
    elif "Alg.2" in k:
        color = colors[0]
    elif "Alg.3" in k:
        color = colors[1]
    elif "Alg.4" in k:
        color = colors[2]
    elif "Alg.5" in k:
        color = colors[3]
    elif "Alg.6" in k:
        color = colors[4]
    elif "Max HV" in k:
        color = colors[8]
#     print(k,"color",color)
    return color

In [5]:

def mohpo_grph(pth,title,up_to_complexity,upper_clip=None,up_to_complexity_y_axis="Hypervolume"):

    df = pd.read_csv(pth,index_col=0)

    if up_to_complexity:
        if up_to_complexity_y_axis=="Hypervolume":
            df1 = df 
        elif up_to_complexity_y_axis=="Log Hypervolume":
            df1 = df.applymap(np.log10)
        elif up_to_complexity_y_axis=="Hypervolume Difference":
            df1 = (- df.drop("max_hv",axis=1).sub(df["max_hv"], axis=0))
        elif up_to_complexity_y_axis=="Log Hypervolume Difference":
            df1 = (- df.drop("max_hv",axis=1).sub(df["max_hv"], axis=0)).applymap(np.log10)
        else:
            raise ValueError(up_to_complexity_y_axis)
    
    else:
        max_hv = df["max_hv"].iloc[0]
        df1 = (max_hv - df.drop("max_hv",axis=1)).applymap(np.log10)
    
#     if upper_clip is not None:
#         df1 = df1.clip(upper=upper_clip)
#         df1 = df1.replace(to_replace=upper_clip, value=np.NaN)

    df2 = df1.rename(columns={
    "max_hv": "Max HV", 
    "sobol": "Sobol", 
    "ehvi": "qNEHVI",
    "parego": "qNParEGO",
    "sobol_hv_list": "Sobol", 
    "ehvi_hv_list": "qNEHVI",
    "parego_hv_list": "qNParEGO",
    "c_angle_rule": "Alg.6",
    "u_angle_rule": "Alg.5",
    "c_gift_wrapping": "Alg.3",
    "u_gift_wrapping": "Alg.2",
    "c_tie_break": "Alg.4"})

    df3 = df2[sorted(df2.columns)]

    # df4 = df3.fillna(value=-1)
    # df4 = df3

    xvalues = df3.index
    data = custom_df_to_dict(df3)
    xlabel="number of visited networks"
    ylabel=(up_to_complexity_y_axis if up_to_complexity else "Log Hypervolume Difference")
    xscale="linear"
    linestyles={k:line_func(k) for k in data.keys()}
    colors={k:color_func(k) for k in data.keys()}
    markers = {k:"" for k in sorted(data.keys())}
    labels = {k:k for k in data.keys()}
    legend_ncol=1
    figsize=(6.0,5.1) # without title : (6.0,4.8)
    new_order_sorted_keys=None
    legend_loc="best"
    axes_labelsize=16

    fig = plot_comparison(
        xvalues,
        data,
        xlabel,
        ylabel,
        xscale,
        linestyles,
        colors,
        markers,
        labels,
        legend_ncol,
        figsize,
        new_order_sorted_keys,
        legend_loc,
        axes_labelsize,
        title
    )
    
    if upper_clip is not None:
        fig.axes[0].set_ylim(0,upper_clip)
    
    return fig

In [6]:
def gen_graphs(
    adjusted_data_folder,min_support,up_to_complexity,upper_clips=None,up_to_complexity_y_axis="Hypervolume"):
    
    if upper_clips is None:
        upper_clips = {"micro_joules_per_pixel":None,"model_bits":None,"params":None}

    for x_axis in ["micro_joules_per_pixel","model_bits","params"]:

        fldr = adjusted_data_folder
        if up_to_complexity:
            fil3 = \
            f"{x_axis}_data_bits_over_data_samples_ax_methods_avgs_adjusted_up_to_complexity_support{min_support}.csv"
        else:
            fil3 = \
            f"{x_axis}_data_bits_over_data_samples_ax_methods_avgs_adjusted_support{min_support}.csv"
        pth = fldr + fil3

        title_map = {
            "micro_joules_per_pixel":"$\SI{}{\mu\joule}$ per pixel",
            "model_bits":"Encoded model bits", 
            "params":"Multiply/adds per pixel"
        }

        fig = mohpo_grph(pth,title_map[x_axis],up_to_complexity,upper_clips[x_axis],up_to_complexity_y_axis)

        fig.savefig(fil3.replace(".csv","_edited_graph.png"), dpi=300)

In [7]:


def gen_graphs_rdc(
    adjusted_data_folder,min_support,up_to_complexity,upper_clips=None,up_to_complexity_y_axis="Hypervolume"):

    if upper_clips is None:
        upper_clips = {"flops":None,"params":None}
    
    for x_axis in ["params","flops"]:
        fldr = adjusted_data_folder
        if up_to_complexity:
            fil3 = f"bpp_loss_mse_loss_{x_axis}_ax_methods_avgs_adjusted_up_to_complexity_support{min_support}.csv"
        else:
            fil3 = f"bpp_loss_mse_loss_{x_axis}_ax_methods_avgs_adjusted_support{min_support}.csv"
        pth = fldr+fil3
        
        title_map = {
            "flops":"FLOPs",
            "params":"Number of Parameters",
        }
        
        fig = mohpo_grph(pth,title_map[x_axis],up_to_complexity,upper_clips[x_axis],up_to_complexity_y_axis)
        fig.savefig(fil3.replace(".csv","_edited_graph.png"), dpi=300)

In [8]:
adjusted_data_folder = "/home/lucas/Documents/perceptronac/scripts/tradeoffs/"

up_to_complexity_y_axis="Hypervolume"

for up_to_complexity in [False,True]:
    for min_support in [10]:

        if up_to_complexity:
            upper_clips = {"micro_joules_per_pixel":None,"model_bits":None,"params":None}
        else:
            upper_clips = {"micro_joules_per_pixel":None,"model_bits":None,"params":None}

        gen_graphs(adjusted_data_folder,min_support,up_to_complexity,upper_clips,up_to_complexity_y_axis)

In [9]:

adjusted_data_folder = "/home/lucas/Documents/perceptronac/scripts/tradeoffs/"

up_to_complexity_y_axis="Hypervolume"

for up_to_complexity in [False,True]:
    for min_support in [10]:

        if up_to_complexity:
            if up_to_complexity_y_axis=="Hypervolume Difference":
                upper_clips = {"flops":0.25*1e10,"params":0.325*1e6}
            elif up_to_complexity_y_axis=="Log Hypervolume Difference":
                upper_clips = {"flops":None,"params":None}
            elif up_to_complexity_y_axis=="Hypervolume":
                upper_clips = {"flops":None,"params":None}
            else:
                raise ValueError(up_to_complexity_y_axis)
        else:
            upper_clips = {"flops":None,"params":None}
        
        gen_graphs_rdc(adjusted_data_folder,min_support,up_to_complexity,upper_clips,up_to_complexity_y_axis)

# Dummy Figure 1

In [10]:
# import matplotlib.pyplot as plt

# y_range = [0.140,0.170]
# x_range = [0.0003,0.0020]

# y_tick_lbls = [0.140,0.145,0.150,0.155,0.160,0.165,0.170]
# # x_tick_lbls = [0.0004,0.0006,0.0008,0.0010,0.0012,0.0014,0.0016,0.0018,0.0020]
# x_tick_lbls = [4000,6000,8000,10000,12000,14000]

# fig = plt.figure(figsize=(6.1,4.64))
# ax = fig.add_subplot(1,1,1)
# ax.plot([0.0004,0.0018],[0.145,0.165])
# ax.set_xlim(x_range[0],x_range[-1])
# ax.set_ylim(y_range[0],y_range[-1])
# ax.set_xticks(list(map(lambda x: x/7864320, x_tick_lbls)))
# ax.set_yticks(y_tick_lbls) 
# ax.set_xticklabels( x_tick_lbls)
# ax.set_yticklabels(map('{0:.3f}'.format,y_tick_lbls))
# ax.set_xlabel("Complexity (encoded model bits)", fontsize=16)
# ax.set_ylabel("Rate (bits per pixel)", fontsize=16)
# fig.tight_layout()
# ax.figure.savefig(f"dummy_fig_1.png", dpi=300)

# Dummy Figure 2

In [11]:
# import matplotlib.pyplot as plt

# y_range = [0.110,0.140]
# x_range = [0.002,0.02]

# y_tick_lbls = [0.110,0.115,0.120,0.125,0.130,0.135,0.140]
# # x_tick_lbls = [0.002,0.004,0.006,0.008,0.01,0.012,0.014,0.016,0.018,0.020]
# x_tick_lbls = [35000,55000,75000,95000,115000,135000]

# fig = plt.figure(figsize=(6.1,4.64))
# ax = fig.add_subplot(1,1,1)
# ax.plot([0.004,0.018],[0.115,0.135])
# ax.set_xlim(x_range[0],x_range[-1])
# ax.set_ylim(y_range[0],y_range[-1])
# ax.set_xticks(list(map(lambda x: x/7864320, x_tick_lbls)))
# ax.set_yticks(y_tick_lbls) 
# ax.set_xticklabels( x_tick_lbls)
# ax.set_yticklabels(map('{0:.3f}'.format,y_tick_lbls))
# ax.set_xlabel("Complexity (encoded model bits)", fontsize=16)
# ax.set_ylabel("Rate (bits per pixel)", fontsize=16)
# fig.tight_layout()
# ax.figure.savefig(f"dummy_fig_2.png", dpi=300)